<a href="https://colab.research.google.com/github/jherna85-pixel/SDS-510/blob/main/Module_4_Project_Web_Scraping_Basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SDS 510: Module 4 Project
## Web Scraping Basics

Jennifer Hernandez

In [ ]:
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

Using one example article, define a function that extracts the information you want from the page: url, author, title, date line, article

In [ ]:
def grabarticle(thisurl):
  thispage = requests.get(thisurl)
  bs = BeautifulSoup(thispage.text,'html.parser')

  title_tag = bs.find('h1')
  title = title_tag.get_text(strip=True) if title_tag else ""

  paragraphs = bs.find_all('p')
  article = '\n\n'.join(x.get_text(strip=True) for x in paragraphs)


  return {'url':thisurl,'title':title,'article':article}


Create an empty dataframe

In [ ]:
df = pd.DataFrame(columns=['url', 'title','article','keyword(s)'])

Create a list of links in the homepage of news.asu.edu; they are under card-title > a > stretched-link > href


In [ ]:
thispage = requests.get('https://news.asu.edu/')
bs = BeautifulSoup(thispage.text,'html.parser')
homepagelinks = bs.find_all('a',class_='stretched-link')

In [ ]:
homepagelinks

[<a class="stretched-link" href="https://news.asu.edu/20251120-science-and-technology-asu-dreamscape-learn-bring-immersive-vr-biology-classrooms-phoenix">ASU, Dreamscape Learn bring immersive VR biology classrooms to Phoenix school district</a>,
 <a class="stretched-link" href="https://news.asu.edu/20251118-science-and-technology-collaboration-key-success-asus-forensic-school-director-says">Collaboration is key to the success of ASU’s forensic school, director says</a>,
 <a class="stretched-link" href="https://news.asu.edu/20251030-university-news-asu-ranked-among-world-best-graduate-employability">ASU named among the world’s best universities for graduate employability</a>,
 <a class="stretched-link" href="https://news.asu.edu/2025022-science-and-technology-asu-computer-scientists-ai-jobs-and-road-ahead">The moment to lead: ASU computer scientists on AI, jobs and the road ahead</a>,
 <a class="stretched-link" href="https://news.asu.edu/20250915-science-and-technology-asu-student-works

In [ ]:
links = []
for eachlink in homepagelinks:
    full_url = eachlink["href"]
    links.append(full_url)


In [ ]:
links

['https://news.asu.edu/20251120-science-and-technology-asu-dreamscape-learn-bring-immersive-vr-biology-classrooms-phoenix',
 'https://news.asu.edu/20251118-science-and-technology-collaboration-key-success-asus-forensic-school-director-says',
 'https://news.asu.edu/20251030-university-news-asu-ranked-among-world-best-graduate-employability',
 'https://news.asu.edu/2025022-science-and-technology-asu-computer-scientists-ai-jobs-and-road-ahead',
 'https://news.asu.edu/20250915-science-and-technology-asu-student-works-protect-african-penguins-local-aquarium',
 'https://news.asu.edu/20251121-arts-humanities-and-education-asu-again-among-top-contributors-teach-america',
 'https://news.asu.edu/20251121-arts-humanities-and-education-elementary-school-teachers-boast-classroom-benefits-after',
 'https://news.asu.edu/20251120-science-and-technology-asu-dreamscape-learn-bring-immersive-vr-biology-classrooms-phoenix',
 'https://news.asu.edu/20251121-health-and-medicine-ais-expanding-role-health-care

Download the article of each link and search for keywords

In [ ]:
keywords = ['medicine','breakthrough','upcoming']

In [ ]:
for link in links:
  if not df.url.str.contains(link).any():
    print("Visiting:", link)
    story = grabarticle (link)
    fullstory = story['article'].lower() #to match case sensitive
    wordfound = [k for k in keywords if k in fullstory]
    if not wordfound:
      continue
    print ('keywords found: ', wordfound)
    new_row = pd.DataFrame([{ #added this as I could not use append
      'keyword(s)': ', '.join(wordfound),
      'url': story['url'],
      'title': story['title'],
      'article': story['article']}])
    df = pd.concat([df, new_row], ignore_index=True) #added this as I could not use append

Visiting: https://news.asu.edu/20251120-science-and-technology-asu-dreamscape-learn-bring-immersive-vr-biology-classrooms-phoenix
Visiting: https://news.asu.edu/20251118-science-and-technology-collaboration-key-success-asus-forensic-school-director-says
keywords found:  ['medicine']
Visiting: https://news.asu.edu/20251030-university-news-asu-ranked-among-world-best-graduate-employability
Visiting: https://news.asu.edu/2025022-science-and-technology-asu-computer-scientists-ai-jobs-and-road-ahead
Visiting: https://news.asu.edu/20250915-science-and-technology-asu-student-works-protect-african-penguins-local-aquarium
Visiting: https://news.asu.edu/20251121-arts-humanities-and-education-asu-again-among-top-contributors-teach-america
Visiting: https://news.asu.edu/20251121-arts-humanities-and-education-elementary-school-teachers-boast-classroom-benefits-after
Visiting: https://news.asu.edu/20251120-science-and-technology-asu-dreamscape-learn-bring-immersive-vr-biology-classrooms-phoenix
Visi

Save to CSV

In [ ]:
df.to_csv('asunewsmatch.csv')